In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import resample
import nltk
from nltk.tokenize import word_tokenize
import pandas as pd
import re
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import re

In [ ]:
fake_df = pd.read_csv("data/Fake.csv")
true_df = pd.read_csv("data/True.csv")

Combining Datasets for Balance Analysis

In [4]:
fake_df['source'] = 'Fake'
true_df['source'] = 'True'

combined_df = pd.concat([fake_df, true_df], axis=0)
df = combined_df.sample(frac=1).reset_index(drop=True)
df = df[['text', 'title', 'source']] 
df.to_csv('C:\\Users\\csalm\\OneDrive\\Documents\\ECE539_final_project\\dataset.csv', index=False)
df

,text,title,source
0,JOHANNESBURG (Reuters) - A violent feud betwee...,Three more cars torched in South African taxi war,True
1,BEIJING (Reuters) - China’s navy will join wil...,Xi says China navy to join U.S.-led 2018 Pacif...,True
2,There s no doubt about it: Trump and his campa...,Time Magazine Cover PERFECTLY Illustrates Wha...,Fake
3,WASHINGTON (Reuters) - U.S. House Speaker Paul...,House Speaker Ryan says he believes upbeat Tre...,True
4,Republicans in Oklahoma just took their war on...,Oklahoma GOP Just Took Their War On Women To ...,Fake
...,...,...,...
44893,Arguably the most popular myth in the Pantheon...,Ted Cruz Continues To Market In Totally Full ...,Fake
44894,DUBAI (Reuters) - Dubai s ruler has pardoned a...,Dubai frees Briton sentenced for touching anot...,True
44895,"Deep down, they just want to assimilate Eight ...",MUSLIM INVASION UPDATE: Gang Members In German...,Fake
44896,(Reuters) - Mississippi’s Republican governor ...,Mississippi governor to join suit against Obam...,True


Source Assessment

In [5]:
df.isna().sum()
df.dropna()

,text,title,source
0,JOHANNESBURG (Reuters) - A violent feud betwee...,Three more cars torched in South African taxi war,True
1,BEIJING (Reuters) - China’s navy will join wil...,Xi says China navy to join U.S.-led 2018 Pacif...,True
2,There s no doubt about it: Trump and his campa...,Time Magazine Cover PERFECTLY Illustrates Wha...,Fake
3,WASHINGTON (Reuters) - U.S. House Speaker Paul...,House Speaker Ryan says he believes upbeat Tre...,True
4,Republicans in Oklahoma just took their war on...,Oklahoma GOP Just Took Their War On Women To ...,Fake
...,...,...,...
44893,Arguably the most popular myth in the Pantheon...,Ted Cruz Continues To Market In Totally Full ...,Fake
44894,DUBAI (Reuters) - Dubai s ruler has pardoned a...,Dubai frees Briton sentenced for touching anot...,True
44895,"Deep down, they just want to assimilate Eight ...",MUSLIM INVASION UPDATE: Gang Members In German...,Fake
44896,(Reuters) - Mississippi’s Republican governor ...,Mississippi governor to join suit against Obam...,True


##### **Combining text and title column, these are our features**

In [6]:
text = (df['title'] + df['text']).astype(str)
text

0        three more cars torched in south african taxi ...
1        xi says china navy to join u.s.-led 2018 pacif...
2        time magazine cover perfectly illustrates what...
3        house speaker ryan says he believes upbeat tre...
4        oklahoma gop just took their war on women to a...
                               ...                        
44893    ted cruz continues to market in totally full o...
44894    dubai frees briton sentenced for touching anot...
44895    muslim invasion update: gang members in german...
44896    mississippi governor to join suit against obam...
44897    fox news host makes herself a total laughingst...
Length: 44898, dtype: object

Cleaning and Lemmatizing Text

In [41]:
def clean_text(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.strip()
    return text

stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
def lemmatize_stopwords(words):
    return [lemmatizer.lemmatize(word) for word in words if word.lower() not in stop_words]

tokens = text.apply(clean_text).apply(word_tokenize).apply(lemmatize_stopwords)
tokens

0        [three, car, torched, south, african, taxi, wa...
1        [xi, say, china, navy, join, u, led, pacific, ...
2        [time, magazine, cover, perfectly, illustrates...
3        [house, speaker, ryan, say, belief, upbeat, tr...
4        [oklahoma, gop, took, war, woman, whole, new, ...
                               ...                        
44893    [ted, cruz, continues, market, totally, full, ...
44894    [dubai, free, briton, sentenced, touching, ano...
44895    [muslim, invasion, update, gang, member, germa...
44896    [mississippi, governor, join, suit, obama, tra...
44897    [fox, news, host, make, total, laughingstock, ...
Length: 44898, dtype: object

Creating Corpus

In [42]:
corpus = tokens.apply(lambda x: ' '.join(x))
corpus

0        three car torched south african taxi warjohann...
1        xi say china navy join u led pacific rim drill...
2        time magazine cover perfectly illustrates happ...
3        house speaker ryan say belief upbeat treasury ...
4        oklahoma gop took war woman whole new level cr...
                               ...                        
44893    ted cruz continues market totally full sh myth...
44894    dubai free briton sentenced touching another m...
44895    muslim invasion update gang member germany tri...
44896    mississippi governor join suit obama transgend...
44897    fox news host make total laughingstock attempt...
Length: 44898, dtype: object

Vectorizing (TF-IDF Vectorization)

In [46]:
vectorizer = TfidfVectorizer(use_idf=True, max_df=0.5, ngram_range=(1, 1))
tfidf = vectorizer.fit_transform(corpus)
X = tfidf
y = df['source']